In [8]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import re
# import spacy
import kagglehub
import string
import re
# import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
from scipy.special import expit
import ast
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import nltk

In [9]:
# Download latest version
path = kagglehub.dataset_download("pes12017000148/food-ingredients-and-recipe-dataset-with-images")
print("Path to dataset files:", path)
recipes = pd.read_csv(path+"/Food Ingredients and Recipe Dataset with Image Name Mapping.csv")
recipes

Path to dataset files: /Users/helenwu/.cache/kagglehub/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images/versions/1


,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten..."
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s..."
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...


In [10]:
null_recs = recipes.copy().drop(columns = 'Image_Name').T.isna().any()
recipes = recipes.drop(index = recipes[null_recs].index).reset_index(drop = True)
recipes["index"] = recipes['Unnamed: 0']
recipes = recipes.drop(columns = ['Unnamed: 0', "Ingredients"])
empty_instr_ind = [index for i, index in zip(recipes['Instructions'], recipes.index) if len(i) < 20]
recipes = recipes.drop(index = empty_instr_ind).reset_index(drop=True)

In [11]:
stop_words = set(stopwords.words('english'))
punctuations =  '"\'\\,<>./?@#$%^&*_~/!()-[]{};:'
custom_stopwords = set(["cup", "cups", "minutes", "oil", "teaspoon", "tablespoon", "ounce", "ounces", "gram", "grams", "minute", "hour", "hours"])
frequent_words = set(['tablespoons', 'large','salt','heat', 'chopped','Add','fresh','sugar','bowl','butter','flour','baking','mixture','f',
 'pan','cream','large', 'chopped', 'Add', 'fresh', 'water', 'medium', 'skillet',  'sauce', 'cook', 'juice', 'cut',  'remaining', 'small','inch', 'finely', 'pot', 'oven', "new", "old", "previous",
                      'pound','sliced','tender','ground','transfer','heavy','teaspoons', 'occasionally', 'leaves','let','add','cover']
)
colors = [
    "red", "blue", "green", "yellow", "orange", "purple", "pink", "brown",
    "black", "white", "gray", "grey", "beige", "cyan", "magenta", "teal",
    "violet", "indigo", "maroon", "turquoise", "lavender", "gold", "silver"
]

stopwords = stop_words.union(custom_stopwords).union(frequent_words).union(colors)

In [12]:
def clean_text(documents):
    cleaned_text = []
    for doc in documents:
        # print(doc)
        doc = doc.lower()
        doc = re.sub(r'\d+', ' ', doc) # Remove Digits
        line = ''.join(char for char in doc if char not in (punctuations) and (char.isalpha() or char == ' '))
        words = line.split()
        words = [word for word in words if word not in stopwords]
        cleaned = ' '.join(words)
        cleaned_text.append(cleaned)
    return cleaned_text

In [13]:
recipes["combined_cleaned"] = clean_text(recipes["Title"] + recipes["Instructions"] )

In [14]:
recipes.head(5)

,Title,Instructions,Image_Name,Cleaned_Ingredients,index,combined_cleaned
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",0,misobutter roast chicken acorn squash panzanel...
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...",1,crispy pepper potatoespreheat line rimmed shee...
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...",2,thanksgiving mac cheeseplace rack middle prehe...
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...",3,italian sausage bread stuffingpreheat rack mid...
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",4,newtons lawstir together hot cocktail shaker d...


In [15]:
recipes['combined_cleaned'] = recipes['combined_cleaned'].str.split()

# Train the SE

In [16]:
from gensim.models import Word2Vec


In [18]:
model = None
def train_Word2Vec(vector_size=100, window=5, min_count=1, epochs=20, workers=4):
  model = Word2Vec(
      sentences=recipes['combined_cleaned'],
      vector_size=vector_size,
      window=window,
      min_count=min_count,
      workers=workers,
      epochs=epochs
  )
  return model
model = train_Word2Vec(vector_size=100, window=5, min_count=1, epochs=20, workers=4)


def compute_avg_log_likelihood(query, doc, epsilon=1e-10):
  similarity_scores = []
  def sigmoid(x):
    return expit(x)
  for query_word in query.split():
      if query_word in model.wv:
          similarities = [model.wv.similarity(query_word, doc_word) for doc_word in doc.split()]
          similarity = sigmoid(max(similarities))
          similarity_scores.append(similarity)
  avg_similarity = np.mean(similarity_scores)
  avg_similarity = max(avg_similarity, epsilon)
  log_likelihood = np.log(avg_similarity)
  return log_likelihood


def execute_search_Word2Vec(query):
  relevances = np.zeros(recipes.shape[0])
  for index, row in recipes.iterrows():
      doc_text = ' '.join(row['combined_cleaned'])
      relevances[index] = compute_avg_log_likelihood(query, doc_text)
  return relevances

In [22]:
from Levenshtein import distance
# Typo correction function
def correct_word(word, vocabulary):
    # Find the closest word in the vocabulary based on edit distance
    closest_word = min(vocabulary, key=lambda x: distance(word, x))
    # Define a threshold for acceptable corrections
    return closest_word if distance(word, closest_word) <= 2 else word

# Preprocess query to handle typos
def preprocess_query(query):
    corrected_query = []
    for word in query.split():
        if word in model.wv.index_to_key:  # If word exists in vocabulary, keep it
            corrected_query.append(word)
        else:  # Otherwise, attempt to correct it
            corrected_query.append(correct_word(word, model.wv.index_to_key))
    return ' '.join(corrected_query)

In [23]:
# typical american food queries test
queries2 = [
    "vegan burger",
    "backyard barbeque",
    "chicken sandwich with american cheese",
    "aisan chiken" # query with typos
]
for query in queries2:

    print(f"QUERY: {query}")
    query = preprocess_query(query)
    relevance_scores = execute_search_Word2Vec(query)

    sorted_indices = np.argsort(relevance_scores)[::-1]

    print("5 most relevant documents:")
    for i in range(5):
        doc_index = sorted_indices[i]
        doc_content = recipes.iloc[doc_index]['Title']
        print(f"Relevance Score: {relevance_scores[doc_index]:.4f}, Title: {doc_content}")

QUERY: vegan burger
5 most relevant documents:
Relevance Score: -0.3133, Title: Big Bhaji Burger
Relevance Score: -0.3567, Title: The Best Vegan Breakfast Sandwich
Relevance Score: -0.3685, Title: Basic Veggie Burgers
Relevance Score: -0.3760, Title: Vegan Niçoise-Style Salad
Relevance Score: -0.3781, Title: Classic Smashed Cheeseburger
QUERY: backyard barbeque
5 most relevant documents:
Relevance Score: -0.4286, Title: Slow-Grilled Leg of Lamb with Mint Yogurt and Salsa Verde
Relevance Score: -0.4290, Title: Padrón Peppers Stuffed with Tetilla Cheese
Relevance Score: -0.4319, Title: Coffee-Rubbed Cheeseburgers with Texas Barbecue Sauce
Relevance Score: -0.4341, Title: Caramelized-Onion Flatbreads with Crème Fraîche
Relevance Score: -0.4342, Title: Root Vegetable Tagine with Sweet Potatoes, Carrots, Turnips, and Spice-Roasted Chickpeas
QUERY: chicken sandwich with american cheese
5 most relevant documents:
Relevance Score: -0.3739, Title: Breaded Chicken Sandwiches with Sesame Rolls (C